In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import os

In [2]:
# Khởi tạo trình duyệt Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") 
options.add_argument("--disable-blink-features=AutomationControlled")  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [3]:
# Đọc file CSV đầu vào
input_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/indata3seealllinks.csv"  
df_input = pd.read_csv(input_file)

In [4]:
# Đường dẫn file đầu ra
output_dir = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv"
output_file = os.path.join(output_dir, "finalindata3seealllinks.csv")

In [5]:
# Danh sách để lưu dữ liệu crawl được
data = []

In [6]:
# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [7]:
# Hàm cuộn xuống cuối trang để tải toàn bộ nội dung
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Chờ tải nội dung
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [8]:
# Hàm định dạng lại văn bản
def format_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    phone_pattern = r'\(?\d{3}\)?\s*-?\s*\d{3}\s*-?\s*\d{4}'
    phone_matches = re.findall(phone_pattern, text)
    for phone in phone_matches:
        phone_clean = re.sub(r'\D', '', phone)
        if len(phone_clean) == 10:
            formatted_phone = f"({phone_clean[:3]}) {phone_clean[3:6]}-{phone_clean[6:]}"
            text = text.replace(phone, formatted_phone)
    email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
    email_matches = re.findall(email_pattern, text)
    for email in email_matches:
        text = text.replace(email, email.strip())
    address_pattern = r'[A-Za-z\s]+,\s*[A-Za-z\s]+,\s*[A-Za-z\s]+,\s*\d{5}'
    address_matches = re.findall(address_pattern, text)
    for address in address_matches:
        formatted_address = address.replace(',', ', ')
        text = text.replace(address, formatted_address)
    text = re.sub(r'\n\s*\n+', '\n', text)
    return text


# Hàm cuộn xuống cuối trang nhanh hơn
def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Giảm thời gian chờ từ 3s xuống 1s


# Duyệt qua từng link trong file CSV
for index, row in df_input.iterrows():
    resume_link = row['resume_link']
    category = row['Category']
    current_id = index + 1
    
    print(f"Đang crawl link: {resume_link} (ID: {current_id})")
    
    try:
        driver.get(resume_link)
        time.sleep(2)  # Giảm thời gian chờ từ 5s xuống 2s
        scroll_to_bottom(driver)  # Cuộn nhanh
        
        # Kiểm tra xem trang có tải được không
        page_source = driver.page_source
        if "Access Denied" in page_source or "403 Forbidden" in page_source or len(page_source) < 1000:
            print(f"Không thể truy cập {resume_link}. HTML: {page_source[:500]}")
            continue
        
        resume_str = ""
        resume_html = ""
        
        # Kiểm tra dạng 1: <section class="jbt-txt-version">
        try:
            jbt_section = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "jbt-txt-version"))
            )
            jbt_container = jbt_section.find_element(By.CLASS_NAME, "jbt-container")
            text_only = jbt_container.find_element(By.CLASS_NAME, "text-only")
            resume_html = jbt_section.get_attribute('outerHTML')
            
            soup = BeautifulSoup(resume_html, 'html.parser')
            text_parts = []
            seen_texts = set()
            
            # Duyệt tuần tự qua các thẻ theo thứ tự xuất hiện trong DOM
            for tag in soup.find_all(['div', 'p', 'h3', 'li']):
                if tag.name == 'div':
                    # Xử lý <strong> trong <div>
                    for strong in tag.find_all('strong'):
                        strong.replace_with(strong.get_text(strip=True))
                    # Chỉ lấy nội dung từ <div> nếu không có thẻ con khác (leaf node)
                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                        text = tag.get_text(separator=" ", strip=True)
                        if text and text not in seen_texts:
                            text_parts.append(text)
                            seen_texts.add(text)
                elif tag.name == 'p':
                    # Xử lý các thẻ con như <strong> và <br>
                    for strong in tag.find_all('strong'):
                        strong.replace_with(strong.get_text(strip=True))
                    for br in tag.find_all('br'):
                        br.replace_with('\n')
                    # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác (leaf node)
                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                        text = tag.get_text(separator=" ", strip=True)
                        if text and text not in seen_texts:
                            text_parts.append(text)
                            seen_texts.add(text)
                elif tag.name == 'h3':
                    # Xử lý <strong> và <a> trong <h3>
                    for strong in tag.find_all('strong'):
                        for a_tag in strong.find_all('a'):
                            a_text = a_tag.get_text(strip=True)
                            strong.replace_with(a_text)
                    # Chỉ lấy nội dung từ <h3> nếu không có thẻ con khác (leaf node)
                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                        text = tag.get_text(strip=True)
                        if text and text not in seen_texts:
                            text_parts.append(text)
                            seen_texts.add(text)
                elif tag.name == 'li':
                    # Xử lý <br> trong <li>
                    for br in tag.find_all('br'):
                        br.replace_with('\n')
                    # Chỉ lấy nội dung từ <li> nếu không có thẻ con khác
                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                        text = tag.get_text(separator=" ", strip=True)
                        if text and text not in seen_texts:
                            text_parts.append(text)
                            seen_texts.add(text)
            
            if text_parts:
                resume_str = "\n".join(text_parts)
                resume_str = format_text(resume_str)
                print("Tìm thấy dạng 1: jbt-txt-version")
            else:
                raise Exception("Không tìm thấy nội dung hợp lệ trong jbt-txt-version")
        
        except Exception:
            # Kiểm tra dạng 2: <div class="container-xs content-wbg p-tb-50 grey-bg txt-cv cta-center">
            try:
                container_xs = WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "div.container-xs.content-wbg.p-tb-50.grey-bg.txt-cv.cta-center[data-format='stm-editor-blade-2']")
                    )
                )
                container = container_xs.find_element(By.CLASS_NAME, "container")
                editor_content = container.find_element(By.CLASS_NAME, "editor-content")
                resume_html = container_xs.get_attribute('outerHTML')
                
                soup = BeautifulSoup(resume_html, 'html.parser')
                text_parts = []
                seen_texts = set()
                
                # Duyệt tuần tự qua các thẻ theo thứ tự xuất hiện trong DOM
                for tag in soup.find_all(['div', 'p', 'li']):
                    if tag.name == 'div':
                        # Chỉ lấy nội dung từ <div> nếu không có thẻ con khác (leaf node)
                        if not any(child.name for child in tag.children if hasattr(child, 'name')):
                            text = tag.get_text(separator=" ", strip=True)
                            # Bỏ qua các thẻ container chính
                            if text and text not in seen_texts and 'container-xs' not in tag.get('class', []) and 'editor-content' not in tag.get('class', []):
                                text_parts.append(text)
                                seen_texts.add(text)
                    elif tag.name == 'p':
                        # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác
                        if not any(child.name for child in tag.children if hasattr(child, 'name')):
                            text = tag.get_text(separator=" ", strip=True)
                            if text and text not in seen_texts:
                                text_parts.append(text)
                                seen_texts.add(text)
                    elif tag.name == 'li':
                        # Chỉ lấy nội dung từ <li> nếu không có thẻ con khác
                        if not any(child.name for child in tag.children if hasattr(child, 'name')):
                            text = tag.get_text(separator=" ", strip=True)
                            if text and text not in seen_texts:
                                text_parts.append(text)
                                seen_texts.add(text)
                
                # Loại bỏ "Build my CV" nếu có
                text_parts = [part for part in text_parts if "Build my CV" not in part]
                
                if text_parts:
                    resume_str = "\n".join(text_parts)
                    resume_str = format_text(resume_str)
                    print("Tìm thấy dạng 2: container-xs content-wbg p-tb-50 grey-bg txt-cv cta-center")
                else:
                    raise Exception("Không tìm thấy nội dung hợp lệ trong container-xs")
            
            except Exception:
                # Kiểm tra dạng 3: <div class="external_wrapper"> với resume-text
                try:
                    external_wrapper = WebDriverWait(driver, 15).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "external_wrapper"))
                    )
                    blade_container = external_wrapper.find_element(By.CLASS_NAME, "blade-container")
                    blade_1 = blade_container.find_element(By.ID, "blade-1")
                    editor_content = blade_1.find_element(By.CLASS_NAME, "editor-content")
                    resume_text = editor_content.find_element(By.CLASS_NAME, "resume-text")
                    resume_html = resume_text.get_attribute('outerHTML')
                    
                    soup = BeautifulSoup(resume_html, 'html.parser')
                    text_parts = []
                    seen_texts = set()
                    
                    # Xóa thẻ <h3>Resume Text</h3> nếu có
                    for h in soup.find_all(['h3', 'h2']):
                        if h.get_text(strip=True) == "Resume Text":
                            h.decompose()
                    
                    # Duyệt tuần tự qua các thẻ theo thứ tự xuất hiện trong DOM
                    for tag in soup.find_all(['p', 'li']):
                        if tag.name == 'p':
                            # Xử lý các thẻ con như <strong>, <b> và <br>
                            for strong in tag.find_all('strong'):
                                strong.replace_with(strong.get_text(strip=True))
                            for b_tag in tag.find_all('b'):
                                # Xử lý <br> trong <b> trước
                                for br in b_tag.find_all('br'):
                                    br.replace_with('\n')
                                b_tag.replace_with(b_tag.get_text(strip=True))
                            for br in tag.find_all('br'):
                                br.replace_with('\n')
                            # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác (leaf node)
                            if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                text = tag.get_text(separator=" ", strip=True)
                                if text and text not in seen_texts:
                                    text_parts.append(text)
                                    seen_texts.add(text)
                        elif tag.name == 'li':
                            # Xử lý <br> trong <li>
                            for br in tag.find_all('br'):
                                br.replace_with('\n')
                            # Chỉ lấy nội dung từ <li> nếu không có thẻ con khác
                            if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                text = tag.get_text(separator=" ", strip=True)
                                if text and text not in seen_texts:
                                    text_parts.append(text)
                                    seen_texts.add(text)
                    
                    if text_parts:
                        resume_str = "\n".join(text_parts)
                        resume_str = format_text(resume_str)
                        print("Tìm thấy dạng 3: external_wrapper với resume-text")
                    else:
                        raise Exception("Không tìm thấy nội dung hợp lệ trong resume-text")
                
                except Exception:
                    # Kiểm tra dạng 4: <div class="wp-block-mprblock-pjt-text-version pjt-text-version">
                    try:
                        text_version = WebDriverWait(driver, 15).until(
                            EC.presence_of_element_located(
                                (By.CSS_SELECTOR, "div.wp-block-mprblock-pjt-text-version.pjt-text-version#see-text-version")
                            )
                        )
                        container = text_version.find_element(By.CLASS_NAME, "container")
                        content_box = container.find_element(By.CLASS_NAME, "content-box")
                        resume_html = text_version.get_attribute('outerHTML')
                        
                        soup = BeautifulSoup(resume_html, 'html.parser')
                        text_parts = []
                        seen_texts = set()
                        
                        # Xóa thẻ <h2> không mong muốn
                        for h in soup.find_all('h2'):
                            if h.get_text(strip=True) == "Accountant Resume Template (Text Version)":
                                h.decompose()
                        
                        # Duyệt tuần tự qua các thẻ theo thứ tự xuất hiện trong DOM
                        for tag in soup.find_all(['p', 'li']):
                            if tag.name == 'p':
                                # Xử lý các thẻ con như <strong> và <br>
                                for strong in tag.find_all('strong'):
                                    strong.replace_with(strong.get_text(strip=True))
                                for br in tag.find_all('br'):
                                    br.replace_with('\n')
                                # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác (leaf node)
                                if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                    text = tag.get_text(separator=" ", strip=True)
                                    if text and text not in seen_texts:
                                        text_parts.append(text)
                                        seen_texts.add(text)
                            elif tag.name == 'li':
                                # Xử lý <br> trong <li>
                                for br in tag.find_all('br'):
                                    br.replace_with('\n')
                                # Chỉ lấy nội dung từ <li> nếu không có thẻ con khác
                                if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                    text = tag.get_text(separator=" ", strip=True)
                                    if text and text not in seen_texts:
                                        text_parts.append(text)
                                        seen_texts.add(text)
                        
                        if text_parts:
                            resume_str = "\n".join(text_parts)
                            resume_str = format_text(resume_str)
                            print("Tìm thấy dạng 4: wp-block-mprblock-pjt-text-version")
                        else:
                            raise Exception("Không tìm thấy nội dung hợp lệ trong wp-block-mprblock-pjt-text-version")
                    
                    except Exception:
                        # Kiểm tra dạng 5: <div class="external_wrapper"> với revealresume
                        try:
                            external_wrapper = WebDriverWait(driver, 15).until(
                                EC.presence_of_element_located((By.CLASS_NAME, "external_wrapper"))
                            )
                            blade_container = external_wrapper.find_element(By.CLASS_NAME, "blade-container")
                            blade_1 = blade_container.find_element(By.ID, "blade-1")
                            editor_content = blade_1.find_element(By.CLASS_NAME, "editor-content")
                            revealresume = editor_content.find_element(By.CLASS_NAME, "revealresume")
                            resume_html = revealresume.get_attribute('outerHTML')
                            
                            soup = BeautifulSoup(resume_html, 'html.parser')
                            text_parts = []
                            seen_texts = set()
                            
                            # Xóa thẻ <h2> hoặc <h3>Resume Text</h2>
                            for h in soup.find_all(['h2', 'h3']):
                                if h.get_text(strip=True) == "Resume Text":
                                    h.decompose()
                            
                            # Duyệt tuần tự qua các thẻ theo thứ tự xuất hiện trong DOM
                            for tag in soup.find_all(['div', 'p', 'h3', 'li']):
                                if tag.name == 'div':
                                    # Chỉ lấy nội dung từ <div> nếu không có thẻ con khác (leaf node)
                                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                        text = tag.get_text(separator=" ", strip=True)
                                        if text and text not in seen_texts and 'revealresume' not in tag.get('class', []) and 'noustar' not in tag.get('class', []):
                                            text_parts.append(text)
                                            seen_texts.add(text)
                                elif tag.name == 'p':
                                    # Xử lý <strong> và <br> trong thẻ <p>
                                    for strong in tag.find_all('strong'):
                                        strong.replace_with(strong.get_text(strip=True))
                                    for br in tag.find_all('br'):
                                        br.replace_with('\n')
                                    # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác (leaf node)
                                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                        text = tag.get_text(separator=" ", strip=True)
                                        if text and text not in seen_texts:
                                            text_parts.append(text)
                                            seen_texts.add(text)
                                elif tag.name == 'h3':
                                    # Chỉ lấy nội dung từ <h3> nếu không có thẻ con khác
                                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                        text = tag.get_text(strip=True)
                                        if text and text not in seen_texts:
                                            text_parts.append(text)
                                            seen_texts.add(text)
                                elif tag.name == 'li':
                                    # Xử lý <br> trong <li>
                                    for br in tag.find_all('br'):
                                        br.replace_with('\n')
                                    # Chỉ lấy nội dung từ <li> nếu không có thẻ con khác
                                    if not any(child.name for child in tag.children if hasattr(child, 'name')):
                                        text = tag.get_text(separator=" ", strip=True)
                                        if text and text not in seen_texts:
                                            text_parts.append(text)
                                            seen_texts.add(text)
                            
                            if text_parts:
                                resume_str = "\n".join(text_parts)
                                resume_str = format_text(resume_str)
                                print("Tìm thấy dạng 5: external_wrapper với revealresume")
                            else:
                                raise Exception("Không tìm thấy nội dung hợp lệ trong revealresume")
                        
                        except Exception:
                            # Kiểm tra dạng 6: <div class="external_wrapper"> với blade-1 wp-editor p-tb-50 (6a hoặc 6b)
                            try:
                                external_wrapper = WebDriverWait(driver, 15).until(
                                    EC.presence_of_element_located((By.CLASS_NAME, "external_wrapper"))
                                )
                                blade_container = external_wrapper.find_element(By.CLASS_NAME, "blade-container")
                                blade_1 = blade_container.find_element(By.ID, "blade-1")
                                editor_content = blade_1.find_element(By.CLASS_NAME, "editor-content")
                                resume_html = editor_content.get_attribute('outerHTML')
                                
                                soup = BeautifulSoup(resume_html, 'html.parser')
                                text_parts = []
                                seen_texts = set()
                                
                                # Loại bỏ các thẻ không mong muốn
                                editor_content_div = soup.find('div', class_='editor-content')
                                for p in editor_content_div.find_all('p'):
                                    if not p.get_text(strip=True):  # Xóa thẻ <p> trống
                                        p.decompose()
                                for p in editor_content_div.find_all('p', style="text-align: center"):
                                    p.decompose()
                                for div in editor_content_div.find_all('div', class_='lottie-blade'):
                                    div.decompose()
                                for div in editor_content_div.find_all('div', class_='img-wrapper'):
                                    div.decompose()
                                for h3 in editor_content_div.find_all('h3'):
                                    if h3.get_text(strip=True) == "Resume Text":
                                        h3.decompose()
                                
                                # Kiểm tra dạng 6a: Nội dung nằm trong thẻ <p>
                                p_tags = editor_content_div.select('p')
                                has_valid_p_content = False
                                if p_tags:
                                    for p_tag in p_tags:
                                        # Xử lý <br> trong thẻ <p>
                                        for br in p_tag.find_all('br'):
                                            br.replace_with('\n')
                                        # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác (leaf node)
                                        if not any(child.name for child in p_tag.children if hasattr(child, 'name')):
                                            text = p_tag.get_text(separator=" ", strip=True)
                                            if text and text not in seen_texts:
                                                text_parts.append(text)
                                                seen_texts.add(text)
                                                has_valid_p_content = True
                                    if has_valid_p_content:
                                        resume_str = "\n".join(text_parts)
                                        resume_str = format_text(resume_str)
                                        print("Tìm thấy dạng 6a: external_wrapper với blade-1 wp-editor p-tb-50 (nội dung trong thẻ p)")
                                    else:
                                        # Chuyển sang kiểm tra dạng 6b nếu không có nội dung hợp lệ trong thẻ <p>
                                        raw_text = editor_content_div.get_text(separator="\n", strip=True)
                                        lines = raw_text.split("\n")
                                        start_collecting = False
                                        for line in lines:
                                            if not start_collecting and re.match(r'^\s*"?[A-Za-z\s,]+?"?\s*$', line):
                                                start_collecting = True
                                            if start_collecting and line.strip() and line.strip() not in seen_texts:
                                                text_parts.append(line.strip())
                                                seen_texts.add(line.strip())
                                        if text_parts:
                                            resume_str = "\n".join(text_parts)
                                            resume_str = format_text(resume_str)
                                            print("Tìm thấy dạng 6b: external_wrapper với blade-1 wp-editor p-tb-50 (nội dung trực tiếp)")
                                        else:
                                            raise Exception("Không có nội dung phù hợp cho dạng 6a hoặc 6b")
                                else:
                                    # Nếu không có thẻ <p>, kiểm tra dạng 6b ngay lập tức
                                    raw_text = editor_content_div.get_text(separator="\n", strip=True)
                                    lines = raw_text.split("\n")
                                    start_collecting = False
                                    for line in lines:
                                        if not start_collecting and re.match(r'^\s*"?[A-Za-z\s,]+?"?\s*$', line):
                                            start_collecting = True
                                        if start_collecting and line.strip() and line.strip() not in seen_texts:
                                            text_parts.append(line.strip())
                                            seen_texts.add(line.strip())
                                    if text_parts:
                                        resume_str = "\n".join(text_parts)
                                        resume_str = format_text(resume_str)
                                        print("Tìm thấy dạng 6b: external_wrapper với blade-1 wp-editor p-tb-50 (nội dung trực tiếp)")
                                    else:
                                        raise Exception("Không có nội dung phù hợp cho dạng 6b")
                            
                            except Exception:
                                # Kiểm tra dạng 7: <div class="external_wrapper"> với blade-1 wp-editor p-tb-50 và Resume Text
                                try:
                                    external_wrapper = WebDriverWait(driver, 15).until(
                                        EC.presence_of_element_located((By.CLASS_NAME, "external_wrapper"))
                                    )
                                    blade_container = external_wrapper.find_element(By.CLASS_NAME, "blade-container")
                                    blade_1 = blade_container.find_element(By.ID, "blade-1")
                                    editor_content = blade_1.find_element(By.CLASS_NAME, "editor-content")
                                    resume_html = editor_content.get_attribute('outerHTML')
                                    
                                    soup = BeautifulSoup(resume_html, 'html.parser')
                                    text_parts = []
                                    seen_texts = set()
                                    
                                    # Loại bỏ các thẻ không mong muốn
                                    editor_content_div = soup.find('div', class_='editor-content')
                                    for p in editor_content_div.find_all('p'):
                                        if not p.get_text(strip=True):  # Xóa thẻ <p> trống
                                            p.decompose()
                                    for div in editor_content_div.find_all('div', class_='img-wrapper'):
                                        div.decompose()
                                    for div in editor_content_div.find_all('div', class_='lottie-blade'):
                                        div.decompose()
                                    for h3 in editor_content_div.find_all('h3'):
                                        if h3.get_text(strip=True) == "Resume Text":
                                            h3.decompose()
                                    
                                    # Duyệt tuần tự qua các thẻ <p> theo thứ tự xuất hiện trong DOM
                                    p_tags = soup.select('.editor-content p')
                                    if not p_tags:
                                        raise Exception("Không có thẻ <p> nào để xử lý dạng 7")
                                    
                                    for p_tag in p_tags:
                                        # Xử lý các thẻ con như <strong>, <a>, và <br>
                                        for strong_tag in p_tag.find_all('strong'):
                                            for a_tag in strong_tag.find_all('a'):
                                                a_text = a_tag.get_text(strip=True)
                                                strong_tag.replace_with(a_text)
                                        for br in p_tag.find_all('br'):
                                            br.replace_with('\n')
                                        # Chỉ lấy nội dung từ <p> nếu không có thẻ con khác (leaf node)
                                        if not any(child.name for child in p_tag.children if hasattr(child, 'name')):
                                            text = p_tag.get_text(separator=" ", strip=True)
                                            if text and text not in seen_texts:
                                                text_parts.append(text)
                                                seen_texts.add(text)
                                    
                                    if not text_parts:
                                        raise Exception("Không tìm thấy nội dung hợp lệ cho dạng 7")
                                    
                                    resume_str = "\n".join(text_parts)
                                    resume_str = format_text(resume_str)
                                    print("Tìm thấy dạng 7: external_wrapper với blade-1 wp-editor p-tb-50 và Resume Text")
                                
                                except Exception as e:
                                    print(f"Không tìm thấy dạng 1, 2, 3, 4, 5, 6 hoặc 7 trong {resume_link}: {e}")
                                    print(f"HTML của trang: {page_source[:500]}")
                                    with open(f"debug_html_{current_id}.html", "w", encoding="utf-8") as f:
                                        f.write(page_source)
                                    print(f"Đã lưu HTML vào debug_html_{current_id}.html")
                                    continue
        
        # Hiển thị nội dung của resume_str sau khi crawl
        print(f"Nội dung resume_str (ID: {current_id}):")
        print(resume_str)
        print("-" * 50)  # Dòng phân cách để dễ đọc
        
        data.append({
            "ID": current_id,
            "resume_str": resume_str,
            "resume_html": resume_html,
            "Category": category
        })
        print(f"Đã crawl thành công: {resume_link} - {category}")
    
    except Exception as e:
        print(f"Lỗi khi crawl link {resume_link}: {e}")
        continue

Đang crawl link: https://www.myperfectresume.com/cv/examples/accounting/trainee (ID: 1)
Tìm thấy dạng 2: container-xs content-wbg p-tb-50 grey-bg txt-cv cta-center
Nội dung resume_str (ID: 1):
Heather Chambers 123 Fake Street, City, State, Zip Code E: email@email.com P: (000) 000-0000 Professional Summary Conscientious accountant trainee with two years of professional experience in an office setting. Assist accountants with preparing financial reports and maintaining financial records. Shadow accountants to learn best practices; quick learner with an eye for detail and a strong interest in the subject matter. Currently enrolled in last year of B.S. in accounting. Skills Understanding of Intuit QuickBooks, Sage 50 Accounting, Yardi Systems Yardi Enterprise, ATX Total Tax Office, and Oracle E-Business Suite Financials Thorough knowledge of statistics, algebra, geometry, calculus, and arithmetic Skilled in administrative procedures and systems, including managing records and files, word p

In [9]:
# Đóng trình duyệt
driver.quit()

In [10]:
# Chuyển dữ liệu thành DataFrame
df_output = pd.DataFrame(data)

In [11]:
df_output.head(5)

,ID,resume_str,resume_html,Category
0,1,"Heather Chambers 123 Fake Street, City, State,...","<div class=""container-xs content-wbg p-tb-50 g...",Accountant Trainee CV
1,2,"Alexander Butler 123 Fake Street, City, State,...","<div class=""container-xs content-wbg p-tb-50 g...",Accounts Officer CV
2,3,"Jessica Francis 123 Fake Street, City, State, ...","<div class=""container-xs content-wbg p-tb-50 g...",Audit Assistant CV
3,4,"Brittany Finn 123 Fake Street, City, State, Zi...","<div class=""p-tb-50 txt-cv container-xs conten...",Certified Public Accountant CV
4,5,"Amy Fenway 123 Fake Street, City, State, Zip C...","<div class=""container-xs content-wbg p-tb-50 g...",Cost Accountant CV


In [12]:
# Lưu vào file CSV
df_output.to_csv(output_file, index=False, encoding="utf-8")
print(f"Dữ liệu đã được lưu vào file: {output_file}")
print(f"Tổng số bản ghi crawl được: {len(df_output)}")

Dữ liệu đã được lưu vào file: D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv\finalindata3seealllinks.csv
Tổng số bản ghi crawl được: 808
